Loading the IMDB dataset

i couldn't initialy open the dataset so i found this online. 

In [1]:
import matplotlib.pyplot as plt
from keras.datasets import imdb
import numpy as np

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old


Using TensorFlow backend.


In [2]:
imdb

<module 'keras.datasets.imdb' from '/home/cardosoo/venv/lib/python3.5/site-packages/keras/datasets/imdb.py'>

In [3]:
len(train_data)

25000

top 10,000 most frequent words

In [5]:
max([max(sequence) for sequence in train_data])

9999

In [6]:
word_index = imdb.get_word_index()

1646592/1641221 [==============================] - 2s 1us/step


In [7]:
word_index

{'compendium': 48458,
 'engineer': 8034,
 "gm's": 54879,
 'looonnnggg': 80846,
 'yeardley': 33750,
 'guests': 5537,
 'dickson': 66768,
 'thimig': 79770,
 'chatham': 58346,
 'heavenly': 8907,
 'editorialised': 83841,
 'opéra': 55817,
 'purporting': 47689,
 '10mil': 74807,
 'chant': 13140,
 'apps': 43525,
 "cheung's": 37465,
 'capas': 83561,
 "sense'": 45661,
 'loather': 84828,
 'papamichael': 53801,
 'shiner': 27555,
 'declaim': 74791,
 'judy': 4421,
 'stomps': 26030,
 'maudlin': 13746,
 'adeptness': 81038,
 'wise\x85': 68303,
 'riker': 18367,
 'fraud': 7862,
 'chemicals': 17278,
 'arvide': 77963,
 'overtone': 36057,
 'peasant': 11086,
 'brylcreem': 87545,
 'philosophize': 50343,
 'edged': 9996,
 'ppfff': 62509,
 'smut': 17392,
 'albertson': 39087,
 'blackburn': 51750,
 'swordsman': 12903,
 'horseshit': 87483,
 "superhero's": 64721,
 "'as": 40357,
 'thereby': 7660,
 "qi's": 64204,
 'redistribute': 78840,
 'storytelling': 2801,
 'nobuhiro': 34751,
 'tripods': 25143,
 'interfernce': 57121

In [8]:
type(word_index)

dict

In [9]:
reverse_word_index = dict([(value, key ) for (key, value) in word_index.items()])

In [10]:
' '.join([reverse_word_index.get(i -3, '?') for i in train_data[0]])

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [11]:
' '.join([reverse_word_index.get(i , '?') for i in train_data[0]])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have t

In [12]:
train_data[5][0:4]

[1, 778, 128, 74]

In [13]:
decoded_review =' '.join([reverse_word_index.get(i -3 , '?') for i in train_data[1]])

In Decoded review the indices are offset by 3, because indices 0,1 and 2 are reserved for "padding", "start of sequence," and "unknown"
Which is why the sequence of words makes more sense than the other

In [14]:
seq = [2,6,3,1]
s = seq[0:3]
s[0] = 99
print(s)
print(seq)

[99, 6, 3]
[2, 6, 3, 1]


indexing of a list is another list, indexing of a numpy array is a view
# note 
there is such a thing a basic and advanced indexing... the intricate difference i haven't explored too much,
but advanced indexing is a copy and basic is a view

In [15]:
seq = np.array([2,6,3,1])
s = seq[0:3]
s[0] = 99
print(s)
print(seq)

[99  6  3]
[99  6  3  1]


creating a one-hot vector

In [16]:
def vectorize_sequences(sequences, dimension = 10000):
    results  = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

In [17]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [18]:
print(x_train[0])

[0. 1. 1. ... 0. 0. 0.]


In [19]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

# Building the network

The model definition

In [20]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


Compiling the model

In [21]:
model.compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])

Setting aside a validation set

In [22]:
x_train.shape
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

train model


In [23]:
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size = 512, verbose = 0,validation_data = (x_val,y_val))

Instructions for updating:
Use tf.cast instead.


In [24]:
history_dict = history.history #contains entire history
history_dict.keys()

dict_keys(['val_acc', 'loss', 'val_loss', 'acc'])

In [25]:
val = zip(history_dict['val_loss'], history_dict['loss'])
for i in val:
    print(i) # val_loss, val_acc

(0.37968397860527037, 0.5084240040461222)
(0.3003729674816132, 0.3004293824036916)
(0.3085316753387451, 0.21790880763530732)
(0.28397776350975035, 0.17504446516036987)
(0.28414924812316894, 0.142692573984464)
(0.3165922412395477, 0.11499823834896088)
(0.31268108854293825, 0.09800445106029511)
(0.38593706359863283, 0.08069882659912109)
(0.36348215651512145, 0.0660659032980601)
(0.3841077935218811, 0.05608599482774734)
(0.41529247183799745, 0.043946201266845064)
(0.45254025416374205, 0.03814133314689001)
(0.4699304045677185, 0.029979005377491316)
(0.5022735030174256, 0.024697823320825894)
(0.5342192699432373, 0.017504934355119864)
(0.5711044822692871, 0.014906830390791099)
(0.6024912521362304, 0.015087408000727495)
(0.6822317755699158, 0.007531567775209745)
(0.6773126829147339, 0.012109525287648042)
(0.6900547019958496, 0.00411029639840126)


Plotting the training and validation loss
# for some reason my kernel dies when i use matplotlib so i used a saved the data and plotted the values some in spyder :<

In [26]:
#note your loss and accuracy graphs may look different because of initial randomizaiton
import pickle
#loss_values = history_dict['loss']
#val_loss_values = history_dict['val_loss']
#epochs = range(1, len(loss_values)+1)
pickle_out = open("imdb_history", "wb")
pickle.dump(history_dict,pickle_out)
pickle_out.close()

In [27]:

#plt.plot(epochs, loss_values, 'bo', label='Training loss') #bo for blue dots
#plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')#b for blue lines
#plt.title('Training and validation loss')
#plt.xlabel('Epochs')
#plt.ylabel('Loss')
#plt.legend()

#plt.show()

Retraining a model from scratch

In [28]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])
history2 = model.fit(x_train, y_train, epochs=4, batch_size = 512)

Epoch 1/4
25000/25000 [==============================] - 1s 49us/step - loss: 0.4750 - acc: 0.8219
Epoch 2/4
25000/25000 [==============================] - 1s 44us/step - loss: 0.2657 - acc: 0.9096
Epoch 3/4
25000/25000 [==============================] - 1s 45us/step - loss: 0.1983 - acc: 0.9294
Epoch 4/4
25000/25000 [==============================] - 1s 44us/step - loss: 0.1677 - acc: 0.9402


In [29]:
results = model.evaluate(x_test, y_test)
print(results)
print(model.metrics_names)

25000/25000 [==============================] - 1s 49us/step
[0.3245329585170746, 0.87312]
['loss', 'acc']


Using train network to generate predictions on new data

In [30]:
model.predict(x_test)

array([[0.13865885],
       [0.99969566],
       [0.28815672],
       ...,
       [0.07008764],
       [0.04285681],
       [0.46925393]], dtype=float32)

# Experiment 1
Build models with 1, 3 and 4 hiddden layers


In [62]:
layerz = [1,3,4]
networks = []
for model in range(len(layerz)):
    networks.append(models.Sequential())
    networks[model].add(layers.Dense(16, activation='relu', input_shape = (10000,)))
    for layer in range(layerz[model]):
        networks[model].add(layers.Dense(16, activation = 'relu'))
    networks[model].add(layers.Dense(1, activation = 'sigmoid'))
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])


In [71]:
len(networks[2].layers),len(networks[1].layers),len(networks[0].layers)

(6, 5, 3)

Compile models

In [73]:
for model in range(len(layerz)):
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])

fit models on validation set

In [77]:
historys = []
for model in range(len(layerz)):
    historys.append(networks[model].fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val)))
    

Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 2s 124us/step - loss: 0.0238 - acc: 0.9968 - val_loss: 0.4952 - val_acc: 0.8687
Epoch 2/4
15000/15000 [==============================] - 2s 120us/step - loss: 0.0198 - acc: 0.9973 - val_loss: 0.4923 - val_acc: 0.8716
Epoch 3/4
15000/15000 [==============================] - 2s 109us/step - loss: 0.0173 - acc: 0.9970 - val_loss: 0.5260 - val_acc: 0.8698
Epoch 4/4
15000/15000 [==============================] - 2s 109us/step - loss: 0.0091 - acc: 0.9995 - val_loss: 0.5672 - val_acc: 0.8620
Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 4s 262us/step - loss: 0.5845 - acc: 0.7339 - val_loss: 0.4309 - val_acc: 0.8625
Epoch 2/4
15000/15000 [==============================] - 2s 127us/step - loss: 0.3237 - acc: 0.8954 - val_loss: 0.3123 - val_acc: 0.8773
Epoch 3/4
15000/15000 [==============================] - 2s 121us/step - loss:

'val_loss', 'val_acc', 'loss', 'acc' for all 3 models

In [86]:
final = len(historys[0].history['val_loss']) -1
for i in range(len(layerz)):
    print(str(i + 1))
    print(historys[i].history['val_loss'][final],historys[i].history['val_acc'][final],historys[i].history['loss'][final],historys[i].history['val_acc'][final])
    

1
0.5671663908481598 0.8619999994277954 0.009095460824668408 0.8619999994277954
2
0.308602543592453 0.8767999999046325 0.1720100563287735 0.8767999999046325
3
0.3320854649066925 0.8785000001907348 0.17009066094557443 0.8785000001907348


4 layers had the most accuraccy but 3 hidden layers had the least loss... not sure why i thought the correlation between validation loss and accuracy would be 1.0. Anyways i'll pick 4 cos i guess accuracy is more important.

# side experiment ... (trying layers with sizes 6 ,7,9)


In [87]:
layerz = [6,8,9]
networks = []
for model in range(len(layerz)):
    networks.append(models.Sequential())
    networks[model].add(layers.Dense(16, activation='relu', input_shape = (10000,)))
    for layer in range(layerz[model]):
        networks[model].add(layers.Dense(16, activation = 'relu'))
    networks[model].add(layers.Dense(1, activation = 'sigmoid'))
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])

for model in range(len(layerz)):
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])
    
historys = []
for model in range(len(layerz)):
    historys.append(networks[model].fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val)))
final = len(historys[0].history['val_loss']) -1
for i in range(len(layerz)):
    print(str(i + 1))
    print(historys[i].history['val_loss'][final],historys[i].history['val_acc'][final],historys[i].history['loss'][final],historys[i].history['val_acc'][final])
    

Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 9s 575us/step - loss: 0.6008 - acc: 0.7203 - val_loss: 0.5071 - val_acc: 0.7944
Epoch 2/4
15000/15000 [==============================] - 3s 169us/step - loss: 0.4104 - acc: 0.8930 - val_loss: 0.3715 - val_acc: 0.8805
Epoch 3/4
15000/15000 [==============================] - 2s 118us/step - loss: 0.2519 - acc: 0.9292 - val_loss: 0.3400 - val_acc: 0.8619
Epoch 4/4
15000/15000 [==============================] - 2s 118us/step - loss: 0.1738 - acc: 0.9423 - val_loss: 0.3407 - val_acc: 0.8816
Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 5s 318us/step - loss: 0.6234 - acc: 0.7055 - val_loss: 0.5211 - val_acc: 0.8564
Epoch 2/4
15000/15000 [==============================] - 2s 127us/step - loss: 0.4374 - acc: 0.8867 - val_loss: 0.4222 - val_acc: 0.8745
Epoch 3/4
15000/15000 [==============================] - 2s 134us/step - loss:

interesting observation... 8 layers had the most accuracy... fuck it imma pick 8 layes


# Experiment 2
trying out different hidden units

In [88]:
units = [32,64,128]
layerz = 8
networks = []
for model in range(len(units)):
    networks.append(models.Sequential())
    networks[model].add(layers.Dense(units[model], activation='relu', input_shape = (10000,)))
    for layer in range(layerz):
        networks[model].add(layers.Dense(units[model], activation = 'relu'))
    networks[model].add(layers.Dense(1, activation = 'sigmoid'))
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])



In [100]:
for model in range(len(units)):
    networks[model].compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])
    
historys = []
for model in range(len(units)):
    historys.append(networks[model].fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val)))
final = len(historys[0].history['val_loss']) -1
for i in range(len(units)):
    print(str(i + 1))
    print(historys[i].history['val_loss'][final],historys[i].history['val_acc'][final],historys[i].history['loss'][final],historys[i].history['acc'][final])
    

Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 9s 595us/step - loss: 0.5785 - acc: 0.7047 - val_loss: 0.3838 - val_acc: 0.8525
Epoch 2/4
15000/15000 [==============================] - 3s 170us/step - loss: 0.3183 - acc: 0.8799 - val_loss: 0.3618 - val_acc: 0.8556
Epoch 3/4
15000/15000 [==============================] - 2s 133us/step - loss: 0.2224 - acc: 0.9203 - val_loss: 0.3367 - val_acc: 0.8696
Epoch 4/4
15000/15000 [==============================] - 2s 132us/step - loss: 0.1723 - acc: 0.9389 - val_loss: 0.5416 - val_acc: 0.8058
Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 6s 395us/step - loss: 0.5443 - acc: 0.7251 - val_loss: 0.3513 - val_acc: 0.8620
Epoch 2/4
15000/15000 [==============================] - 3s 172us/step - loss: 0.2826 - acc: 0.8891 - val_loss: 0.2900 - val_acc: 0.8837
Epoch 3/4
15000/15000 [==============================] - 3s 170us/step - loss:

In [101]:
final = len(historys[0].history['val_loss']) -1
for i in range(len(units)):
    print(str(i + 1))
    print(historys[i].history['val_loss'][final],historys[i].history['val_acc'][final],historys[i].history['loss'][final],historys[i].history['acc'][final])
    

1
0.5415851084709168 0.8057999996185303 0.17232653064727782 0.9389333331743877
2
0.38390573854446414 0.8803999998092651 0.13915774693489075 0.9484000000635783
3
0.28638739318847656 0.8872000001907349 0.11825140884717306 0.9564000001271565


the more the hidden units the better the model... Therefore i shall choose 128 hidden units


# Experiment 3
different loss function: mse


In [104]:
units = 128
layerz = 8

network = models.Sequential()
network.add(layers.Dense(units, activation='relu', input_shape = (10000,)))
for layer in range(layerz):
    network.add(layers.Dense(units, activation = 'relu'))
network.add(layers.Dense(1, activation = 'sigmoid'))
network.compile(optimizer = 'rmsprop', loss= 'mse', metrics = ['accuracy'])

In [106]:
history = network.fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val))


Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 8s 543us/step - loss: 0.1897 - acc: 0.7271 - val_loss: 0.1162 - val_acc: 0.8501
Epoch 2/4
15000/15000 [==============================] - 4s 249us/step - loss: 0.0808 - acc: 0.8912 - val_loss: 0.1021 - val_acc: 0.8594
Epoch 3/4
15000/15000 [==============================] - 4s 248us/step - loss: 0.0568 - acc: 0.9260 - val_loss: 0.0961 - val_acc: 0.8706
Epoch 4/4
15000/15000 [==============================] - 4s 256us/step - loss: 0.0335 - acc: 0.9591 - val_loss: 0.0921 - val_acc: 0.8855


AttributeError: 'list' object has no attribute 'history'

In [108]:
final = len(history.history['val_loss']) -1
print(history.history['val_loss'][final],
      history.history['val_acc'][final],
      history.history['loss'][final],
      history.history['acc'][final])


0.09209611726999282 0.8855000003814697 0.03346091506977876 0.9590666665712992


binary_crossentropy:0.28638739318847656 0.8872000001907349 0.11825140884717306 0.9564000001271565 is better


# Experiment 4
different activation function: tanh


In [112]:
network = models.Sequential()
network.add(layers.Dense(units, activation='tanh', input_shape = (10000,)))
for layer in range(layerz):
    network.add(layers.Dense(units, activation = 'tanh'))
network.add(layers.Dense(1, activation = 'sigmoid'))
network.compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])

history = network.fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val))
final = len(history.history['val_loss']) -1
print(history.history['val_loss'][final],
      history.history['val_acc'][final],
      history.history['loss'][final],
      history.history['acc'][final])


Train on 15000 samples, validate on 10000 samples
Epoch 1/4
15000/15000 [==============================] - 10s 642us/step - loss: 0.6770 - acc: 0.6402 - val_loss: 0.3417 - val_acc: 0.8632
Epoch 2/4
15000/15000 [==============================] - 5s 325us/step - loss: 0.3350 - acc: 0.8683 - val_loss: 0.2859 - val_acc: 0.8839
Epoch 3/4
15000/15000 [==============================] - 5s 353us/step - loss: 0.2382 - acc: 0.9080 - val_loss: 0.4118 - val_acc: 0.8413
Epoch 4/4
15000/15000 [==============================] - 5s 359us/step - loss: 0.2053 - acc: 0.9212 - val_loss: 0.3554 - val_acc: 0.8517
0.3554300384998322 0.851699999332428 0.20534951910972596 0.9211999997456869


relu:0.28638739318847656 0.8872000001907349 0.11825140884717306 0.9564000001271565 is better

# final Experiment
evaluation using testing dataset

In [ ]:
network = models.Sequential()
network.add(layers.Dense(units, activation='relu', input_shape = (10000,)))
for layer in range(layerz):
    network.add(layers.Dense(units, activation = 'relu'))
network.add(layers.Dense(1, activation = 'sigmoid'))
network.compile(optimizer = 'rmsprop', loss= 'binary_crossentropy', metrics = ['accuracy'])

history = network.fit(partial_x_train, 
                                         partial_y_train, epochs=4, #i choose 4 epochs because in the above model it was best
                                         batch_size = 512, 
                                         validation_data = (x_val,y_val))
final = len(history.history['val_loss']) -1
print(history.history['val_loss'][final],
      history.history['val_acc'][final],
      history.history['loss'][final],
      history.history['acc'][final])

results = model.evaluate(x_test, y_test)
print(results)
print(model.metrics_names)